### Notebook utilizado para tratamento da base de dados do SCR - Sistema de Informações de Créditos do Bacen.

#### Os arquivos foram baixados no portal de dados abertos do Bacen, disponível no link: https://dadosabertos.bcb.gov.br/dataset/scr_data
#### Em seguida, foram extraídos na pasta local E:/Apps/Dados/SCR
#### Serão utilizados os dados relativos ao mês de dezembro de 2022, mesmo período das informações da RAIS.
##### Os arquivos não foram incluídos no projeto por serem muito grandes.

In [1]:
# Importando bibliotecas
import pandas as pd
import requests
import json

In [2]:
# Definindo as colunas que serão utilizadas.
columns = ['data_base','uf','tcb','cliente','cnae_secao','cnae_subclasse','porte','modalidade','numero_de_operacoes','carteira_ativa']

# Importando a base de dados do SCR referente a Dezembro de 2022.
d = pd.read_csv('E:/Apps/Dados/SCR/planilha_2022/planilha_202212.csv', delimiter=';', usecols=columns, low_memory=False)

# Definindo a base de dados com as informações de Pessoas Jurídicas.
df = d[d.cliente == 'PJ'].copy()

##### A coluna 'cnae_subclasse' está configurada com as descrições da CNAE, mas precisaremos das informações dos códigos da CNAE.
##### Será criado um arquivo json que permita converter as descrições em códigos, conforme o padrão CNAE - Subclasses 2.3 do IBGE.

In [3]:
# Removendo a parte inicial 'PJ - ' de cnae_subclasse.
df.cnae_subclasse = df.cnae_subclasse.apply(lambda x: x[5:])

# Criando cnae_sub com as descrições sem duplicações.
cnae_sub = df.cnae_subclasse.unique()

In [4]:
# Pegando as informações da CNAE Subclasses no IBGE.
r = requests.get('https://servicodados.ibge.gov.br/api/v2/cnae/subclasses')
subclasses = json.loads(r.content)

# Criando um DataFrame com as subclasses
dfsub = pd.DataFrame(subclasses)

In [5]:
# Criando o dicionário cnaes no formato: {"descricao": código,...}
cnaes = {}
c = 0
s = 0
for d in cnae_sub:
    try:
        id = dfsub.classe[dfsub.descricao == d.upper()].iloc[0]['id']
        s += 1   
    except:
        id = 0
        c += 1
    cnaes[d] = id
print('Erros: ', c)
print('Sucesso: ', s)

Erros:  113
Sucesso:  1128


##### Infelizmente há divergências em algumas descrições em cnae_subclasse e nas descrições fornecidas pelo IBGE.
##### Totos os erros foram resolvidos após alguns ajustes em cnae_subclasse e verificação manual no link: https://cnae.ibge.gov.br/

In [8]:
# Salvando o arquivo json sem erros
#with open("../assets/cnaes.json", "w", encoding="utf-8") as arquivo:
#    json.dump(cnaes, arquivo, ensure_ascii=False, indent=4)

##### Utilizando o arquivo cnaes.json

In [6]:
# Abrindo o arquivo novamente, caso seja necessário.
with open("../assets/cnaes.json", "r", encoding="utf-8") as arquivo:
    cnaes = json.load(arquivo)

In [7]:
# Função para converter descrição por código da CNAE.
erro = []
def get_idcnaes(descricao):
    try:
        descricao = str(descricao)
        id = cnaes[descricao]
    except:
        id = ''
        if len(descricao)>0:
            erro.append(descricao)
    return id

In [8]:
# Criando coluna com os códigos.
df['ID CNAE'] = df.cnae_subclasse.apply(lambda x: get_idcnaes(x))

# Importante verificar se ocorreram erros.
if len(erro)>0:
    print('Há erros para corrigir:')
    erro
else:
    print('Sem erros.')

Sem erros.


##### Importante considerar que na base de dados do SCR, quando a combinação CNAE Subclasse x UF x Porte resulta em 5 ou menos CNPJs no cadastro da RFB é reportado apenas o CNAE Seção. [Ver metodolia do SCR.data](https://www.bcb.gov.br/content/estabilidadefinanceira/scr/scr.data/scr_data_metodologia.pdf)
##### Como precisaremos do CNAE Subclasse para os fins desse projeto, devemos verificar a dimensão desses casos.

In [11]:
# Convertendo carteira_ativa em float.
df.carteira_ativa = df.carteira_ativa.apply(lambda x: x.replace(',','.')).astype('float')
# Total da carteira_ativa
t = df.carteira_ativa.sum()
# Total da carteira_ativa SEM a CNAE Subclasse
tsem = df.carteira_ativa[(df.cnae_subclasse == '') | (df.cnae_subclasse == '-')].sum()
# Total da carteira_ativa COM a CNAE Subclasse
tcom = df.carteira_ativa[(df.cnae_subclasse != '') & (df.cnae_subclasse != '-')].sum()
print('Total: {0} | Sem subclasse: {1} | Com subclasse: {2}'.format(t,tsem,tcom))
print('Sem Subclasse representa {0}% do total'.format(((tsem*100)/t).round(2)))

Total: 2080771273012.8503 | Sem subclasse: 102493143528.53 | Com subclasse: 1978278129484.3198
Sem Subclasse representa 4.93% do total


##### Em termos globais, o montante da carteira_ativa sem CNAE Subclasse é inferior a 5% do total.
##### Vamos verificar qual o percentual por estado.

In [13]:
# Criando coluna com total da carteira_ativa sem CNAE Subclasse por uf.
df['carteira_ativa_sem_sub'] = df[df['ID CNAE'] == ''].groupby(['uf'])['carteira_ativa'].transform('sum')
# Criando DataFrame agregado por ur para verificação.
dfsemsubs = df[['uf','carteira_ativa','carteira_ativa_sem_sub']].groupby(['uf'], as_index=False).agg({'carteira_ativa': 'sum', 'carteira_ativa_sem_sub': 'first'})
# Adicionando coluna com percentual da carteira_ativa_sem_sub por uf.
dfsemsubs['%'] = dfsemsubs.carteira_ativa_sem_sub*100/dfsemsubs.carteira_ativa
dfsemsubs[['uf','%']]

,uf,%
0,AC,25.591993
1,AL,6.831319
2,AM,9.009896
3,AP,17.218464
4,BA,10.690224
5,CE,15.103630
6,DF,4.444875
7,ES,7.493176
8,GO,4.661370
9,MA,7.000124


##### Podemos notar que o percentual é bem significativo em estados como Acre, Roraima, Amapá e outros, enquanto assume os menores valores no Rio de Janeiro, São Paulo e Paraná. Esse resultado aparenta ser condizente com a realidade dos estados e o critério utilizado na metodologia para criação dessa base de dados.
##### Os resultados futuros do projeto devem considerar essa caracteristica da base de dados do SCR.

In [14]:
# Salvando a base de dados do SCR
df.to_pickle('../assets/scr122022.pkl',  compression='gzip')